In [77]:
import copy as cp
import math as mt
import numpy as np
import scipy as sc
import pandas as pd

def rmat(nr,nc): return(np.matrix(np.random.rand(nr,nc)))
def hey(s): 
    from scipy.sparse import triu
    return np.matrix(triu(rmat(s,s)).toarray())

class parameters:
    def __init__(self,size):
        self.size=size
        self.mu=np.matrix(np.zeros((size,1)))
        self.F=np.matrix(np.zeros((size,size)))
        self.Q=np.matrix(np.zeros((size,size)))
        self.R=np.matrix(np.zeros((1,1)))
        self.beta00=np.matrix(np.zeros((size,1)))
        self.P00=np.matrix(np.zeros((size,size)))
    def randomize(self):
        s=self.size
        self.mu=rmat(s,1)
        self.F=rmat(s,s)
        Q=hey(s)
        self.Q=Q.transpose()*Q
        self.R=rmat(1,1)
        self.beta00=rmat(s,1)
        P=hey(s)
        self.P00=P.transpose()*P

class thedata:
    def __init__(self,size):
        self.y=np.matrix(np.zeros((1,1)))
        self.x=np.matrix(np.zeros((1,size)))

class a_filter:
    def __init__(self,thefile):
        data=pd.read_csv(thefile)
        
        nr,nc=np.shape(data)
        nx=nc-1
        self.intercept=False
        response=input('Please enter "No" if you do not want an intercept:')
        if (response!="No"):
            nx=nx+1
            self.intercept=True
        self.size=nx
        self.T=nr  
        self.parameters=parameters(self.size)
        self.obs=[]
        for t in range(self.T):
            newobs=an_obs(t,self)
            y=np.matrix(data.iloc[t,0])
            x=np.matrix(data.iloc[t,1:])
            if self.intercept:x=np.concatenate((np.matrix(1),x),axis =1)
            newobs.data.y=y
            newobs.data.x=x
            self.obs=self.obs+[newobs]
            
    def run(self):
        for t in range(self.T):
            self.obs[t].predict()
            self.obs[t].update()               

class an_obs:
    def __init__(self,t,fltr):
        self.fltr=fltr
        self.t=t
        size=self.fltr.size
        self.data=thedata(size)
        self.prediction=predictions(size)
        self.updating=updating(size)
        
    def predict(self):
        if self.t == 0: 
            b00=self.fltr.parameters.beta00
            P00=self.fltr.parameters.P00
        else: 
            b00=self.fltr.obs[self.t-1].updating.beta
            P00=self.fltr.obs[self.t-1].updating.P
        F = self.fltr.parameters.F
        x = self.data.x
        self.prediction.beta=self.fltr.parameters.mu+F*b00
        self.prediction.P=F*P00*np.transpose(F)+self.fltr.parameters.Q
        self.prediction.eta=self.data.y-self.data.x*self.prediction.beta
        self.prediction.f=x*self.prediction.P*x.transpose()+self.fltr.parameters.R
        
    def update(self):
        P=self.prediction.P
        x=self.data.x
        K=P*x.transpose()*np.linalg.inv(self.prediction.f)
        self.updating.K=K
        self.updating.beta=self.prediction.beta+K*self.prediction.eta
        self.updating.P=P-K*x*P

class predictions:
    def __init__(self,size):
        self.beta=np.matrix(np.zeros((size,1)))
        self.P=np.matrix(np.zeros((size,size)))
        self.eta=np.matrix(np.zeros((1,1)))
        self.f=np.matrix(np.zeros((1,1)))

class updating:
    def __init__(self,size):
        self.beta=np.matrix(np.zeros((size,1)))
        self.P=np.matrix(np.zeros((size,size)))
        self.K=np.matrix(np.zeros((size,1)))      

In [78]:
ibm_model = a_filter('IBM.csv')

Please enter "No" if you do not want an intercept:yes


In [79]:
ibm_model.obs[7].data.x

matrix([[ 1.    , -0.0284, -0.0083, -0.0323]])

In [80]:
ibm_model.parameters.randomize()

In [81]:
ibm_model.run()

In [82]:
ibm_model.obs[7].prediction.P

matrix([[1.6107408 , 2.27072516, 1.4324167 , 3.50405617],
        [2.27072516, 4.02790027, 2.62157492, 5.29515203],
        [1.4324167 , 2.62157492, 2.15576761, 3.87914467],
        [3.50405617, 5.29515203, 3.87914467, 9.41601813]])